# 像numpy一样使用Tensorflow
## 张量和操作
- 可以使用tf.constant1()操作创建张量。

In [1]:
# 两行三列浮点数矩阵的张量
import tensorflow as tf
tf.constant([[1.,2.,3.],[4.,5.,6.]])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [2]:
# 标量
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

就像ndarray一样tf.Tensor具有形状和数据类型

In [3]:
t=tf.constant([[1.,2.,3.],[4.,5.,6.]])
t.shape

TensorShape([2, 3])

In [4]:
t.dtype

tf.float32

索引的工作方式非常像numpy。

In [5]:
t[:,1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

最重要的是可以使用各种张量操作

In [6]:
t+10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [7]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [8]:
t@tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

## 张量和numpy
张量可以与NumPy配合使用:你可以用numpy数组创建张量，反之亦然。你甚至可以将Tensorflow操作应用于Numpy数组，将numpy操作应用于张量。

In [9]:
import numpy as np
a=np.array([2.,4.,5.])

In [10]:
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [11]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [12]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [13]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

注意默认情况下numpy使用64位精度，而TensorFlow使用32位精度。因此当你从numpy数组创建张量时，请确保设置dtype=tf.float32。

## 类型转换
Tensorflow不会自动执行任何类型转换，如果你对不兼容类型的张量操作，会引发异常。

In [14]:
# 进行类型转换
t2=tf.constant(40.,dtype=tf.float64)

In [15]:
tf.constant(2.0)+tf.cast(t2,tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

## 变量
到目前为止，我们看到的tf.Tensor值是不变的，无法修改它们。我们需要的是题tf.Variable

In [16]:
v=tf.Variable([[1.,2.,3.],[4.,5.,6.]])

In [17]:
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

## 其它数据结构
- 稀疏张量 tf.SparseTensor
- 张量数组 tf.TensorArray
- 不规则张量 tf.RaggedTensor
- 字符串张量 tf.string
- 集合 tf.sets
- 队列 tf.queue

# 定制模型和训练算法
## 自定义损失函数
来写一个huber损失函数。![huber loss](./huber.png)

In [18]:
# 此时假设阈值δ为1
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss  = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)
# where是三元表达式

应仅使用tf操作。可以在编译的时候使用该损失函数model.compile(loss=huber_fn, ...)

## 保存和加载具有自定义组件的模型

In [19]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [20]:
X_train_scaled.shape

(11610, 8)

In [21]:
import keras
model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=[8]),
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(1),
])

In [22]:
model.compile(loss=huber_fn, optimizer=keras.optimizers.Nadam(), metrics=["mae"])

In [23]:
model.fit(X_train_scaled,y_train,epochs=2,validation_data=(X_valid,y_valid))

Epoch 1/2
363/363 [==============================] - 13s 2ms/step - loss: 0.9688 - mae: 1.3701 - val_loss: 223.1692 - val_mae: 223.6692
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.2114 - mae: 0.5050 - val_loss: 193.3801 - val_mae: 193.8801


In [24]:
model.save('my_model_with_a_custom_loss.h5')

一般而言，当加载包含自定义对象的模型时，需要将名称映射到对象上。

In [25]:
model=keras.models.load_model('my_model_with_a_custom_loss.h5',custom_objects={'huber_fn':huber_fn})

我们创建一个可以设定不同阈值的huber损失函数：

In [26]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold*tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [27]:
model.compile(loss=create_huber(2.0),optimizer=keras.optimizers.Nadam(),metrics=['mae'])

不幸的是当你保存模型的时候不会保存阈值。这意味着在加载模型时必须指定阈值。

In [28]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 2s 2ms/step - loss: 0.2354 - mae: 0.5039 - val_loss: 0.2501 - val_mae: 0.4947
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.2213 - mae: 0.4873 - val_loss: 0.2217 - val_mae: 0.4912


In [29]:
model.save("my_model_with_a_custom_loss_threshold_2.h5")

In [30]:
model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5",
                                custom_objects={"huber_fn": create_huber(2.0)})

可以通过创建keras.losses.Loss类的子类，然后实现其get_config()方法来解决此问题。

In [31]:
class HuberLoss(keras.losses.Loss):
    def __init__(self,threshold=1.0,**kwargs):
        self.threshold=threshold
        super().__init__(**kwargs)
    def call(self,y_true,y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold*tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config=super().get_config()
        return {**base_config,'threshold':self.threshold}

然后你可以在编译模型时使用此类的任何实例。

In [32]:
model.compile(loss=HuberLoss(2.0),optimizer=keras.optimizers.Nadam(),metrics=['mae'])

In [33]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 2s 2ms/step - loss: 0.2214 - mae: 0.4932 - val_loss: 0.2070 - val_mae: 0.4734
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.2165 - mae: 0.4882 - val_loss: 0.2098 - val_mae: 0.4770


当你保存模型的时候阈值会一起保存，当你加载模型的时候只需要将类名映射到函数本身即可。

In [34]:
model.save("my_model_with_a_custom_loss_class.h5")

In [35]:
model = keras.models.load_model("my_model_with_a_custom_loss_class.h5",
                                custom_objects={"HuberLoss": HuberLoss})

## 自定义激活函数和正则化约束

自定义激活函数softplus()

In [36]:
def my_softplus(z):
    return tf.math.log(tf.exp(z)+1.0)

自定义初始化glorot_normal()

In [37]:
def my_glorot_initializer(shape,dtype=tf.float32):
    stddev=tf.sqrt(2./(shape[0]+shape[1]))
    return tf.random.normal(shape,stddev=stddev,dtype=dtype)

自定义l1正则化

In [38]:
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01*weights))

确保权重为正的自定义约束keras.contraints.nonneg()

In [39]:
def my_positive_weights(weights):
    return tf.where(weights<0.,tf.zeros_like(weights),weights)

然后就可以使用这些函数了。

In [40]:
layer = keras.layers.Dense(30, activation=my_softplus, kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer, kernel_constraint=my_positive_weights)

激活函数将应用于此Dense层的输出，其结果将传递下一层。层的权重将使用初始化程序返回的值进行初始化。在每个训练步骤中，权重将传递给正则化函数以计算正则化损失，并将其添加到主要损失中以得到用于训练的最终损失。最后将在每个训练步骤结束后调用约束函数，并将层的权重替换为约束权重。

如果函数具有需要与模型一起保存的超参数，你需要继承适当的类。注意你必须要实现call()（损失、层和模型）或者__ call __()（正则化、初始化、约束）。

In [41]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self,factor):
        self.factor=factor
    def __call__(self,weights):
        return tf.reduce_sum(tf.abs(self.factor*weights))
    def get_config(self):
        return {'factor':self.factor}

## 自定义评价指标
大多数情况下，定义一个指标函数与定义一个损失函数大致相同。

In [42]:
model.compile(optimizer=keras.optimizers.Nadam(),loss='mse',metrics=[create_huber(2.0)])

训练的每个批次，Keras都会计算该指标并跟踪自轮次开始和以来的均值。

In [43]:
percision=keras.metrics.Precision()

In [44]:
percision([0,1,1,1,0,1,0,1],[1,1,0,1,0,1,0,1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

前面是标签，后面是预测。正例1中5个有4个被预测正确，所以精确度为80%

自定义部分不做重点说明。详细见书。

## 自定义层

首先某些层没有权重，例如keras.layers.Flatten()。要创建一个不带权重的层，最简单的选择就是编写一个函数并将其包装在Lambda层中。

In [45]:
# 以下层对它的输入应用指数函数。
exponential_layer=keras.layers.Lambda(lambda x : tf.exp(x))

要创建自定义的有状态层(即具有权重的层),你需要创建keras.layers.Layer类的子类。

In [46]:
# 以下实现了Dense层的简化版本。
class MyDense(keras.layers.Layer):
    def __init__(self,units,activation=None,**kwargs):
        super().__init__(**kwargs)
        self.units=units
        self.activation=keras.activations.get(activation)
    def build(self,batch_input_shape):
        self.kernel=self.add_weight(name='kernel',shape=[batch_input_shape[-1],self.units],initializer='glorot_normal')
        self.bias=self.add_weight(name='bias',shape=[self.units],initializer='zeros')
        super().build(batch_input_shape) # 必须放在最后
    def call(self,X):
        return self.activationa(X@self.kernel+self.bias)
    def compute_output_shape(self,batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1]+[self.units])
    def get_config(self):
        base_config=super().get_config()
        return {**base_config,'units':self.units,'activation':keras.activations.serialize(self.activation)}

build(self, input_shape)：此方法可用于创建依赖于输入形状的权重，使用 add_weight()。 __ call __() 将通过调用 build() 自动构建层（如果尚未构建）。

每一个方法的具体解释见书。不看也能理解。通常可以省略compute_output_shape()方法。

要创建一个具有多个输入的层，call()方法的参数应该包含所有输入的元组，同样，compute_output_shape()方法的参数应该是一个包含每个输入批处理形状的元组。

In [47]:
class MyMultiLayer(keras.layers.Layer):
    def call(self,X):
        X1,X2=X
        return [X1+X2,X1*X2,X1/X2]
    def compute_output_shape(self,batch_input_shape):
        b1,b2=batch_input_shape
        return [b1,b1,b1]

如果你的层在训练期间和测试期间需要有不同的行为，则必须将训练参数添加到call()方法并使用此参数来决定要做什么。

In [48]:
class MyGaussianNoise(keras.layers.Layer):
    def __init__(self,stddev,**kwargs):
        super().__init__(**kwargs)
        self.stddev=stddev
    def call(self,X,training=None):
        if training:
            noise=tf.random.normal(tf.shape(X),stddev=self.stddev)
            return X+noise
        else:
            return X
    def compute_output_shape(self,batch_input_shape):
        return batch_input_shape

## 自定义模型
![custom model](./custom_model.png)

**要实现此模型，首先最好要创建一个ResidualBlock层，因为我们要创建几个相同的块。**

In [49]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self,n_layers,n_neurons,**kwargs):
        super().__init__(**kwargs)
        self.hidden=[keras.layers.Dense(n_neurons,activation='elu',kernel_initializer='he_normal') for _ in range(n_layers)]
    def call(self,inputs):
        Z=inputs
        for layer in self.hidden:
            Z=layer(Z) # 输入按层传递
        return inputs+Z # 拟合图中输出

这层比较特殊，因为包含其它的层。这会由keras透明处理。

接下来我们用model子类api来实现该模型。

In [50]:
class ResidualRegressor(keras.Model):
    def __init__(self,output_dim,**kwargs):
        super().__init__(**kwargs)
        self.hidden1=keras.layers.Dense(30,activation='elu',kernel_initializer='he_normal')
        self.block1=ResidualBlock(2,30)
        self.block2=ResidualBlock(2,30)
        self.out=keras.layers.Dense(output_dim)
    def call(self,inputs):
        Z=self.hidden1(inputs)
        for _ in range(1+3):
            Z=self.block1(Z) # 先走一次，再回来走三次
        Z=self.block2(Z)
        return self.out(Z)

如果你还希望能够使用save()方法保存模型和load_model()加载模型，则必须在两个ResidualBlock类和ResidualRegressor类中实现get_config()方法。（就像以前一样）

## 基于模型内部的损失和指标
例如我们构建一个自定义回归MLP模型，该模型由5个隐藏层和一个输出层的堆栈组成。此自定义模型还将在上部隐藏层的顶部有辅助输出。与该辅助输出相关的损失称之为重建损失。详见17章。

In [51]:
class ReconstructingRegressor(keras.Model):
    def __init__(self,output_dim,**kwargs):
        super().__init___(**kwargs)
        self.hidden=[keras.layers.Dense(30,activation='selu',kernel_initializer='lecun_normal') for _ in range(5)]
        self.out=keras.layers.Dense(output_dim)
    def build(self,batch_input_shape):
        n_inputs=batch_input_shape[-1]
        self.reconstruct=keras.layers.Dense(n_inputs)
        super().build(batch_input_shape)
    def call(self,inputs):
        Z=inputs
        for layer in self.hidden:
            Z=layer(Z)
        reconstruction=self.reconstruct(Z)
        recon_loss=tf.reduce_mean(tf.square(reconstruction-inputs))
        self.add_loss(0.05*recon_loss)
        return self.out(Z)

- 构造函数创建具有5个隐藏层和一个密集输出层的DNN
- build()方法这是你定义权重的地方。在这里创建一个密集层，该层用于重建模型的输入。
- call()方法处理所有5个隐藏层的输入，然后将结果传递到重建层，从而产生重构。
- 然后call()方法计算重建损失，并使用add_loss()方法将其添加到模型的损失列表中。
- 最后call()方法将隐藏层的输出传递到输出层将其输出。

## 使用自动微分计算梯度
让我们考虑一个简单的函数。

In [52]:
def f(w1,w2):
    return 3*w1**2+2*w1*w2

该函数对w1的偏导数是6 * w1+2 * w2，对w2的偏导数是2 * w1。但是对于神经网络手工分析找到偏导数几乎是不可能完成的任务。

一种方案是通过在调整相应的参数时测量函数输出的变化来计算每个偏导数的近似值。

In [53]:
w1,w2=5,3
eps=1e-6
(f(w1+eps,w2)-f(w1,w2))/eps

36.000003007075065

In [54]:
(f(w1,w2+eps)-f(w1,w2))/eps

10.000000003174137

看起来不错，但这只是一个近似值。重要的是每个参数至少要调用一次f()，这种方法对于大型神经网络来说很棘手。因此，我们应该使用自动微分。

In [55]:
w1,w2=tf.Variable(5.),tf.Variable(3.)

In [56]:
with tf.GradientTape() as tape:
    z=f(w1,w2)

In [57]:
gradients=tape.gradient(z,[w1,w2])

In [58]:
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

我们首先定义两个变量w1和w2，然后创建一个tf.GradientTape上下文,该上下文将自动记录涉及变量的每个操作。最后我们要求该tape针对两个变量w1和w2计算结果z的梯度。

调用tape的gradient()方法后，tape会立即被自动擦除，因此如果你尝试两次调用gradient()，则会出现异常。

In [60]:
with tf.GradientTape() as tape:
    z=f(w1,w2)
    
gradients=tape.gradient(z,w1)
# gradients=tape.gradient(z,w2)

如果你需要多次调用gradient()方法后，tape必须具有持久性，并在每次使用完该tape后将其删除以释放资源。

In [61]:
with tf.GradientTape(persistent=True) as tape:
    z=f(w1,w2)

gradients=tape.gradient(z,w1)
gradients=tape.gradient(z,w2)
del tape

后面部分略，详见书。

## 自定义训练循环
**一般情况下，不要自定义训练循环，使用默认的fit()方法。**

首先我们建立一个简单的模型。

In [62]:
l2_reg = keras.regularizers.l2(0.05)
model = keras.models.Sequential([keras.layers.Dense(30, activation='elu', kernel_initializer='he_normal',
                                kernel_regularizer=l2_reg), keras.layers.Dense(1, kernel_regularizer=l2_reg)])

接下来我们创建一个小函数,从训练集中随机采样一批实例。

In [63]:
def random_batch(X,y,batch_size=32):
    idx=np.random.randint(len(X),size=batch_size)
    return X[idx],y[idx]

让我们定义一个函数来显示训练状态，包括步数和总步数，从轮次开始以来的平均损失，和其它指标。

In [64]:
def print_status_bar(iteration,total,loss,metrics=None):
    metrics=' - '.join(["{}:{:.4f}".format(m.name,m.result()) for m in [loss]+(metrics or [])])
    end='' if iteration<total else '\n'
    print('\r{}/{} - '.format(iteration,total)+metrics,end=end)

后面也省略。详细见书。

# TensorFlow函数和图

In [65]:
def cube(x):
    return x**3

In [66]:
cube(2)

8

In [67]:
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

现在我们要tf.function()将此python函数转换成Tensorflow函数。

In [68]:
tf_cube=tf.function(func=cube)

In [69]:
tf_cube

然后就可以像使用python函数一样使用此tf函数。

In [70]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [71]:
tf_cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

在后台，tf.function()分析了cube()函数执行的计算，并生成等效的计算图。

另外我们也可以用tf.function作为修饰器。

In [72]:
@tf.function
def tf_cube(x):
    return x**3

也可以用TF函数的python_function属性使用原python函数。

In [73]:
tf_cube.python_function(2)

8